In [4]:
import pandas as pd
import os

In [5]:
os.getcwd()

'C:\\Users\\student\\Documents\\Daikoku_Python'

In [6]:
df = pd.read_excel('9_spam.xlsx')

In [7]:
df

,mail,label
0,me free lottery,1
1,free get free you,1
2,you free scholarship,0
3,free to contact me,0
4,you won award,0
5,you ticket lottery,1


# Naive Bayes Classifier

In [9]:
from collections import defaultdict
import numpy as np
import math

In [10]:
k = 0.5 # for laplace smoothing : 확률이 0인 경우를 곱하여 전체 식이 무너지는 것을 막기위함

input_file = df
traing_set = np.array(input_file)
print(traing_set)

[['me free lottery' 1]
 ['free get free you' 1]
 ['you free scholarship' 0]
 ['free to contact me' 0]
 ['you won award' 0]
 ['you ticket lottery' 1]]


### 토큰 빈도수 및 문서별 토큰 수 계산

In [28]:
# 범주에 속하는 토큰 수 세기 1(예, 긍정) 0 (예. 부정)
doccnt1 = 0
doccnt0 = 0

# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])
for doc, point in traing_set:
    words = doc.split()
    for word in words:
        if point == 1:
            wordfreq[word][0] += 1
        else:
            wordfreq[word][1] += 1

for key, (cnt1, cnt0) in wordfreq.items():
    if cnt1 > 0 :
        doccnt1 += 1
    if cnt0 > 0 :
        doccnt0 += 1
        
print(wordfreq)

defaultdict(<function <lambda> at 0x000000000961B840>, {'me': [1, 1], 'free': [3, 2], 'lottery': [2, 0], 'get': [1, 0], 'you': [2, 2], 'scholarship': [0, 1], 'to': [0, 1], 'contact': [0, 1], 'won': [0, 1], 'award': [0, 1], 'ticket': [1, 0]})


## 코드 이해

##### numpy의 이용  
 - 판다스는 데이터를 출력하면 시리즈형식으로 출력
 - numpy로 바꿔서 for문을 사용하면 원래 형식의 데이터로 출력 가능

In [22]:
b = np.arange(12).reshape(3,-1)
b

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [27]:
for x, y, z, w in b:
    print(x)
    print(y)
    print(z)
    print(w)

0
1
2
3
4
5
6
7
8
9
10
11


In [19]:
for doc, point in traing_set:
    print(doc)
    print(type(doc))
    print(point)
    print(type(point))

me free lottery
<class 'str'>
1
<class 'int'>
free get free you
<class 'str'>
1
<class 'int'>
you free scholarship
<class 'str'>
0
<class 'int'>
free to contact me
<class 'str'>
0
<class 'int'>
you won award
<class 'str'>
0
<class 'int'>
you ticket lottery
<class 'str'>
1
<class 'int'>


- defaultdict(lambda : [0, 0])

1. defaultdict의 사용

In [36]:
a = defaultdict(lambda : [0, 0])
a['b'][0] += 1
a['c'][1] += 1
print(a['b'])
print(a['c'])
print(a)

[1, 0]
[0, 1]
defaultdict(<function <lambda> at 0x000000000961BD08>, {'b': [1, 0], 'c': [0, 1]})


2. .items

In [44]:
print(a)
print(a.items())
for x, (y, z) in a.items():
    print(x, y, z)

defaultdict(<function <lambda> at 0x000000000961BD08>, {'b': [1, 0], 'c': [0, 1]})
dict_items([('b', [1, 0]), ('c', [0, 1])])
b 1 0
c 0 1


### Traing : 토큰별 조건부 확률 계산

In [13]:
wordprobs = defaultdict( lambda : [0, 0])
for key, (cnt1, cnt0) in wordfreq.items() :
    wordprobs[key][0] = (cnt1 + k) / (doccnt1 + 2*k)
    wordprobs[key][1] = (cnt0 + k) / (doccnt0 + 2*k)

print(wordprobs)

defaultdict(<function <lambda> at 0x000000000961B7B8>, {'me': [0.21428571428571427, 0.16666666666666666], 'free': [0.5, 0.2777777777777778], 'lottery': [0.35714285714285715, 0.05555555555555555], 'get': [0.21428571428571427, 0.05555555555555555], 'you': [0.35714285714285715, 0.2777777777777778], 'scholarship': [0.07142857142857142, 0.16666666666666666], 'to': [0.07142857142857142, 0.16666666666666666], 'contact': [0.07142857142857142, 0.16666666666666666], 'won': [0.07142857142857142, 0.16666666666666666], 'award': [0.07142857142857142, 0.16666666666666666], 'ticket': [0.21428571428571427, 0.05555555555555555]})


### 신규 텍스트가 주어졌을 때 확률 계산

In [14]:
doc = 'free lottery'
tokens = doc.split()

# 초기값은 모두 0으로 처리
log_prob1 = log_prob0 = 0.0

# 모든 단어에 대해 반복
for word, (prob1, prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 += math.log(prob1)
        log_prob0 += math.log(prob0)
        
    else:
        log_prob1 += math.log(1.0 - prob1)
        log_prob0 += math.log(1.0 - prob0)
        
log_prob1 += math.log(doccnt1/len(wordfreq))
log_prob0 += math.log(doccnt0/len(wordfreq))

prob1 = math.exp(log_prob1)
prob0 = math.exp(log_prob0)

print(doc)
print('스팸확률 : {}%'.format(prob1 / (prob1 + prob0) * 100))
print('정상확률 : {}%'.format(prob0 / (prob1 + prob0) * 100))


free lottery
스팸확률 : 89.64758762435878%
정상확률 : 10.352412375641222%
